mounting gdrive

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


importing libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

reading csv file

In [4]:
path = "/gdrive/My Drive/ML:March2020/data/"
data = pd.read_csv(path+"paint_mnist.csv",header=None)
data.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,255,255,255,255,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,255,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
data.shape

(3599, 785)

In [7]:
cols=data.shape[1]
print("columns={}".format(cols))

columns=785


In [9]:
y=np.array(data.iloc[:,:1])
x=np.array(data.iloc[:,1:cols])

In [13]:
x.shape,y.shape

((3599, 784), (3599, 1))

In [14]:
b=np.unique(y)
b

array([0, 1, 2, 3])

In [15]:
classes=b.size
classes

4

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=2)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((2699, 784), (900, 784), (2699, 1), (900, 1))

In [18]:
x=np.array(x_train)
y=np.array(y_train)

Inserting a column at the front which is all 1's

In [19]:
rows=x.shape[0]
x=np.insert(x,0,values=np.ones(rows),axis=1)
x.shape

(2699, 785)

In [20]:
params=x.shape[1]
all_theta=np.zeros((classes,params))
all_theta.shape

(4, 785)

In [21]:
def sigmoid(z):
  return 1/(1+np.exp(-z))

In [23]:
def computeCost(theta,x,y):
  theta=np.matrix(theta)
  x=np.matrix(x)
  y=np.matrix(y)
  first=np.multiply(-y,np.log(sigmoid(x*theta.T)))    # formula part 1 J(theta) = -ylog(g(x*theta transpose))
  second=np.multiply((1-y),np.log(1-sigmoid(x*theta.T)))   # formula part 2 J(theta) = -(1-y)log(1-g(x*theta transpose))
  cost=np.sum(first-second)    # formula J(theta)
  cost=cost/(2*len(x))       #constant 1/2m
  return cost

In [24]:
cost=computeCost(all_theta,x,y)
print("cost={}".format(cost))

cost=1.3862943611198906


In [124]:
learning_rate=0.48   # best learning rate
def gradient(theta,x,y):
  theta=np.matrix(theta)
  x=np.matrix(x)
  y=np.matrix(y)
  error=sigmoid(x*theta.T)-y      # formula for gradient -> error = sigmoid(g(x*theta transpose))-y
  grad=((x.T*error)/len(x))       # formula for gradient -> (1/m)*(x transpose * error) 
  grad=grad*learning_rate         # gradient multiplied with learning rate
  return grad

In [125]:
from scipy.optimize import minimize
def one_vs_all(x,y,classes,new_theta):
  params=x.shape[1]
  rows=y.shape[0]
  for i in range(0,classes):
    theta=new_theta[i,:]
    y_i=np.array([1 if label==i else 0 for label in y])
    y_i=np.reshape(y_i,(rows,1))
    fmin=minimize(fun=computeCost,x0=theta,args=(x,y_i),method="TNC",jac=gradient)
    new_theta[i,:]=fmin.x
  return new_theta

In [126]:
new_theta=one_vs_all(x,y,classes,all_theta)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in multiply
  


In [127]:
computeCost(new_theta,x,y)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in multiply
  


nan

In [128]:
def predict_all(x,theta):
  x=np.matrix(x)
  theta=np.matrix(theta)
  h=sigmoid(x*theta.T)
  h_argmax=np.argmax(h,axis=1)
  return h_argmax,h

In [129]:
y_pred,h=predict_all(x,new_theta)

In [130]:
y_pred

matrix([[2],
        [3],
        [1],
        ...,
        [0],
        [1],
        [1]])

In [131]:
def acc(y_pred,y):
  correct=[1 if a==b else 0 for (a,b) in zip(y_pred,y)]
  accuracy=(sum(correct))/(len(correct))
  return accuracy

In [132]:
accuracy=acc(y_pred,y)
print("training data accuracy={}%".format(accuracy*100)) 

training data accuracy=98.11041126343089%


In [133]:
t_x=np.array(x_test)
t_y=np.array(y_test)
t_rows=t_x.shape[0]
t_x=np.insert(t_x,0,values=np.ones(t_rows),axis=1)

In [134]:
t_y_pred,h=predict_all(t_x,new_theta)

In [135]:
print("test data accuracy={}%".format(acc(t_y_pred,t_y)*100))

test data accuracy=77.11111111111111%


In [104]:
from PIL import Image
from PIL import ImageOps
from IPython.display import display
Im=Image.open('/gdrive/My Drive/ML:March2020/data/pic.png')
display(Im)

In [105]:
Im=Im.convert('L')
Im=ImageOps.invert(Im)
display(Im)

In [106]:
Im=np.array(Im)
Im=np.ravel(Im)
Im.shape

(784,)

In [107]:
Im=Im.reshape(1,-1)

In [108]:
input_x=np.insert(Im,0,1,axis=1)

In [109]:
input_x.shape

(1, 785)

In [110]:
pred,h=predict_all(input_x,new_theta)
print('confidense matrix={}'.format(h))
print('predicted class is:{}'.format(pred.item()))
print('confidense ={}'.format(((h[0,pred])*100).item()))

confidense matrix=[[8.28965361e-05 9.98335225e-01 1.09263238e-03 2.53954200e-06]]
predicted class is:1
confidense =99.83352253030048
